In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmcv.runner import load_checkpoint
from tqdm.notebook import tqdm
import torch
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.6.0].
device:[cuda:0].


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/swin/cascade_mask_rcnn_swin_base2.py')

PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train_all.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'

cfg.data.samples_per_gpu = 3

cfg.seed=42
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/swin+cascade+small'
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

In [3]:
import os
from mmcv.runner import load_checkpoint
model = build_detector(cfg.model)
checkpoint_path = os.path.join('./work_dirs/swin+cascade','cascade_mask_rcnn_swin_small_patch4_window7.pth')
checkpoint = load_checkpoint(model, checkpoint_path)

2021-05-20 02:28:45,685 - mmdet - INFO - load model from: /home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/work_dirs/swin+cascade/swin_base_patch4_window7_224_22k.pth
2021-05-20 02:28:47,137 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: norm.weight, norm.bias, head.weight, head.bias, layers.0.blocks.1.attn_mask, layers.1.blocks.1.attn_mask, layers.2.blocks.1.attn_mask, layers.2.blocks.3.attn_mask, layers.2.blocks.5.attn_mask, layers.2.blocks.7.attn_mask, layers.2.blocks.9.attn_mask, layers.2.blocks.11.attn_mask, layers.2.blocks.13.attn_mask, layers.2.blocks.15.attn_mask, layers.2.blocks.17.attn_mask

missing keys in source state_dict: norm0.weight, norm0.bias, norm1.weight, norm1.bias, norm2.weight, norm2.bias, norm3.weight, norm3.bias



Use load_from_local loader
The model and loaded state dict do not match exactly

size mismatch for backbone.patch_embed.proj.weight: copying a param with shape torch.Size([96, 3, 4, 4]) from checkpoint, the shape in current model is torch.Size([128, 3, 4, 4]).
size mismatch for backbone.patch_embed.proj.bias: copying a param with shape torch.Size([96]) from checkpoint, the shape in current model is torch.Size([128]).
size mismatch for backbone.patch_embed.norm.weight: copying a param with shape torch.Size([96]) from checkpoint, the shape in current model is torch.Size([128]).
size mismatch for backbone.patch_embed.norm.bias: copying a param with shape torch.Size([96]) from checkpoint, the shape in current model is torch.Size([128]).
size mismatch for backbone.layers.0.blocks.0.norm1.weight: copying a param with shape torch.Size([96]) from checkpoint, the shape in current model is torch.Size([128]).
size mismatch for backbone.layers.0.blocks.0.norm1.bias: copying a param with shape torc

In [4]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=5.22s)
creating index...
index created!


In [5]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-20 02:28:58,798 - mmdet - INFO - Start running, host: ubuntu@nipa2021-1083, work_dir: /home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/work_dirs/swin+cascade+small
2021-05-20 02:28:58,799 - mmdet - INFO - workflow: [('train', 1)], max: 15 epochs


Done (t=1.42s)
creating index...
index created!


/home/ubuntu/CUAI_2021/Advanced_Won_Cho/code/Swin-Transformer-Object-Detection/mmdet/core/evaluation/eval_hooks.py:111: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
2021-05-20 02:30:00,533 - mmdet - INFO - Epoch [1][50/1091]	lr: 9.890e-06, eta: 5:35:41, time: 1.235, data_time: 0.109, memory: 22007, loss_rpn_cls: 0.6526, loss_rpn_bbox: 0.0840, s0.loss_cls: 2.4300, s0.acc: 8.0221, s0.loss_bbox: 0.1122, s0.loss_mask: 1.1321, s1.loss_cls: 1.2824, s1.acc: 3.0391, s1.loss_bbox: 0.0153, s1.loss_mask: 0.4884, s2.loss_cls: 0.5469, s2.acc: 23.2201, s2.loss_bbox: 0.0017, s2.loss_mask: 0.2777, loss: 7.0232, grad_norm: 61.5901
2021-05-20 02:30:59,660 - mmdet - INFO - Epoch [1][100/1091]	lr: 1.988e-05, eta: 5:27:36, time: 1.183, data_time: 0.042, memory: 22090, loss_rpn_cls: 0.1828, loss_rpn_bbox: 0.0533, s0.loss_cls: 1.0823, s0.acc: 88.3516, s0.loss_bbox: 0.2585, s0.loss_mask: 0.8121, s1.loss_cls: 0.5289, s1.acc: 87.8060, s1.l

2021-05-20 02:48:28,585 - mmdet - INFO - Epoch [1][950/1091]	lr: 1.000e-04, eta: 5:16:21, time: 1.261, data_time: 0.049, memory: 23442, loss_rpn_cls: 0.0665, loss_rpn_bbox: 0.0420, s0.loss_cls: 0.3028, s0.acc: 91.1406, s0.loss_bbox: 0.3004, s0.loss_mask: 0.3636, s1.loss_cls: 0.1483, s1.acc: 91.2542, s1.loss_bbox: 0.1212, s1.loss_mask: 0.1638, s2.loss_cls: 0.0678, s2.acc: 92.0532, s2.loss_bbox: 0.0423, s2.loss_mask: 0.0786, loss: 1.6973, grad_norm: 7.7796
2021-05-20 02:49:31,926 - mmdet - INFO - Epoch [1][1000/1091]	lr: 1.000e-04, eta: 5:15:46, time: 1.267, data_time: 0.064, memory: 23442, loss_rpn_cls: 0.0626, loss_rpn_bbox: 0.0509, s0.loss_cls: 0.3029, s0.acc: 90.7956, s0.loss_bbox: 0.3147, s0.loss_mask: 0.3621, s1.loss_cls: 0.1472, s1.acc: 90.8819, s1.loss_bbox: 0.1216, s1.loss_mask: 0.1603, s2.loss_cls: 0.0679, s2.acc: 91.4986, s2.loss_bbox: 0.0434, s2.loss_mask: 0.0727, loss: 1.7063, grad_norm: 6.9757
2021-05-20 02:50:36,819 - mmdet - INFO - Epoch [1][1050/1091]	lr: 1.000e-04, eta:

...

2021-05-20 09:24:29,878 - mmdet - INFO - Epoch [15][950/1091]	lr: 1.000e-06, eta: 0:03:00, time: 1.285, data_time: 0.051, memory: 23519, loss_rpn_cls: 0.0275, loss_rpn_bbox: 0.0272, s0.loss_cls: 0.1740, s0.acc: 94.2070, s0.loss_bbox: 0.2142, s0.loss_mask: 0.1990, s1.loss_cls: 0.0797, s1.acc: 94.5774, s1.loss_bbox: 0.0866, s1.loss_mask: 0.0890, s2.loss_cls: 0.0385, s2.acc: 94.6724, s2.loss_bbox: 0.0332, s2.loss_mask: 0.0411, loss: 1.0098, grad_norm: 5.6063
2021-05-20 09:25:32,886 - mmdet - INFO - Epoch [15][1000/1091]	lr: 1.000e-06, eta: 0:01:56, time: 1.260, data_time: 0.047, memory: 23519, loss_rpn_cls: 0.0267, loss_rpn_bbox: 0.0234, s0.loss_cls: 0.1619, s0.acc: 94.6250, s0.loss_bbox: 0.2159, s0.loss_mask: 0.1976, s1.loss_cls: 0.0758, s1.acc: 95.1677, s1.loss_bbox: 0.0835, s1.loss_mask: 0.0866, s2.loss_cls: 0.0365, s2.acc: 95.2770, s2.loss_bbox: 0.0320, s2.loss_mask: 0.0382, loss: 0.9781, grad_norm: 5.7956
2021-05-20 09:26:38,043 - mmdet - INFO - Epoch [15][1050/1091]	lr: 1.000e-06, e

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 3.1 task/s, elapsed: 208s, ETA:     0s

2021-05-20 09:31:05,464 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.14s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=7.48s).
Accumulating evaluation results...
DONE (t=0.96s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.523
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.668
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.556
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.209
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.517
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.772
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.628
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.628
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.628
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-20 09:31:18,889 - mmdet - INFO - Now best checkpoint is epoch_15.pth.Best bbox_mAP_50 is 0.6680
2021-05-20 09:31:18,925 - mmdet - INFO - Epoch(val) [15][1091]	bbox_mAP: 0.5230, bbox_mAP_50: 0.6680, bbox_mAP_75: 0.5560, bbox_mAP_s: 0.2090, bbox_mAP_m: 0.5170, bbox_mAP_l: 0.7720, bbox_mAP_copypaste: 0.523 0.668 0.556 0.209 0.517 0.772
